In [6]:
# Import packages
import datetime
import geopandas as gpd
import holidays
# import july
import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt
from dbfread import DBF
# import seaborn as sns
import shutil
import time
# Import CEA modules
import cea.config
import cea.api 
from cea.utilities.dbf import *
from cea.utilities.epwreader import epw_reader
from cea.datamanagement.archetypes_mapper import archetypes_mapper
from cea.demand.schedule_maker.schedule_maker import main as schedule_maker
from cea.demand.demand_main import main as demand_main
# from cea.resources.radiation_daysim.radiation_main import main as radiation_main

DAYSIM_BIN_DIRECTORY = os.path.join(os.getcwd(), 'CEA_model/0_daysim_binaries/darwin')
project_folder = os.path.join(os.getcwd(), 'CEA_model')

In [7]:
zone_gdf = gpd.read_file(
    os.path.join(project_folder, 'Block0202023', '2020', 'inputs', 'building-geometry', 'zone.shp')) 
list_buildings = zone_gdf["Name"].tolist() 
import cea.config
config = cea.config.Configuration()
config.multiprocessing = True
for year in ['2020_de', '2021_de', '2022_de']:
    config.project = os.path.join(project_folder, 'Block0202023')
    config.scenario_name = year
    config.archetypes_mapper.buildings = list_buildings

In [8]:
config.archetypes_mapper.buildings

['B1000', 'B1001', 'B1002', 'B1003', 'B1004', 'B1005']

In [9]:
vars(config.archetypes_mapper)

{'name': 'archetypes-mapper',
 'config': <cea.config.Configuration at 0x232a26155e0>,
 'parameters': {'input-databases': <Parameter archetypes-mapper:input-databases=['comfort', 'architecture', 'air-conditioning', 'internal-loads', 'supply', 'schedules']>,
  'buildings': <Parameter archetypes-mapper:buildings=['B1000', 'B1001', 'B1002', 'B1003', 'B1004', 'B1005']>}}

In [11]:
# Possible combinations, AB_E, AB_E_NR and "AB_E_AR"
archetypes = ["AB_E" , "AB_E_NR", "AB_E_AR"] 

In [17]:
new_scenario_name = 'modified-occupancy-schedules' # the new scenario name
for archetype in archetypes:
    year = "2020"
    path_to_new_scenario = os.path.join(project_folder, 'Block0202023', f'{year}_{archetype}') # create the destination path for copying the baseline scenario
    path_to_baseline = os.path.join(project_folder, 'Block0202023', year)
    shutil.copytree(path_to_baseline, path_to_new_scenario)
    for record in DBF(r'CEA_model\Block0202023\2021\inputs\building-properties\typology.dbf'):
        record["STANDARD"] = archetype
    config.demand.buildings = list_buildings
    demand_main(config)


In [22]:
cea.api.demand(scenario=path_to_new_scenario, weather = r'C:\Users\felix\Programmieren\cea_berlin\auxillary\AMY_2010_2022_2020.epw')

AttributeError: module 'cea' has no attribute 'api'